# Chapter 8: Constructors & Object Initialization

Every object begins its life through a **constructor** – a special method that is called when an instance of a class or struct is created. Constructors ensure that an object starts in a valid state by initializing its fields and properties. In this chapter, you’ll learn everything about constructors, from the default parameterless constructor to advanced techniques like constructor chaining, static constructors, and the modern `required` modifier. You’ll also discover how **object initializers** provide a flexible way to set properties after construction.

By the end of this chapter, you’ll be able to control object creation with precision, enforce mandatory data, and write cleaner initialization code.

---

## 8.1 Default and Parameterized Constructors

### The Default Constructor

If you don’t define any constructor in a class, the C# compiler automatically provides a **default parameterless constructor**. It initializes all fields to their default values (e.g., `0` for numeric types, `null` for reference types, `false` for `bool`).

```csharp
public class Person
{
    public string Name;
    public int Age;
}

// The compiler adds: public Person() { }

Person p = new Person(); // Name is null, Age is 0
```

Once you add any custom constructor, the compiler no longer generates the default parameterless constructor. If you still need one, you must explicitly define it.

### Parameterized Constructors

Often you want to require certain data when creating an object. You do this by defining a constructor with parameters.

```csharp
public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }

    // Parameterized constructor
    public Person(string name, int age)
    {
        Name = name;
        Age = age;
    }
}

// Usage
Person p = new Person("Alice", 30); // Must provide name and age
```

Now you cannot create a `Person` without providing a name and age (unless you add a parameterless constructor).

### Multiple Constructors (Overloading)

A class can have multiple constructors with different parameter lists, just like method overloading.

```csharp
public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }

    public Person() : this("Unknown", 0) { } // calls another constructor (see chaining)

    public Person(string name) : this(name, 0) { }

    public Person(string name, int age)
    {
        Name = name;
        Age = age;
    }
}
```

This gives flexibility: you can create a person with just a name, or with both, or with defaults.

### Constructor for Structs

Structs also have constructors, but with some differences:
- A struct always has an implicit parameterless constructor that zero‑initializes all fields (you cannot replace it with a custom parameterless constructor until C# 10, which allows explicit parameterless constructors).
- In a struct constructor, you must assign a value to all fields before the constructor exits (unless it’s a parameterless constructor that you define explicitly in C# 10+).

```csharp
public struct Point
{
    public int X, Y;

    public Point(int x, int y)
    {
        X = x;
        Y = y;
    }
}
```

With C# 10 and later, you can define explicit parameterless constructors for structs:

```csharp
public struct Point
{
    public int X, Y;

    public Point()
    {
        X = 0;
        Y = 0;
    }
}
```

But the default zero‑initialization still happens before the constructor body runs, so you can’t prevent fields from being zero‑initialized.

---

## 8.2 Constructor Chaining (`this()`)

Often constructors share common initialization logic. Instead of duplicating code, you can have one constructor call another using `this()`.

```csharp
public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }
    public string City { get; set; }

    public Person(string name, int age) : this(name, age, "Unknown") { }

    public Person(string name, int age, string city)
    {
        Name = name;
        Age = age;
        City = city;
    }
}
```

The `: this(name, age, "Unknown")` syntax calls the three‑parameter constructor, passing the default city. This ensures that all construction logic flows through one primary constructor.

**Rules:**
- The `this()` call must be the first thing in the constructor body (before any other code).
- You can chain to any other constructor in the same class.
- This pattern is especially useful when you have validation or complex initialization in one place.

### Primary Constructors (C# 12)

C# 12 introduced **primary constructors** for classes and structs (previously only available for records). You can define parameters directly in the class declaration, and they are in scope throughout the class.

```csharp
public class Person(string name, int age)
{
    public string Name { get; } = name;
    public int Age { get; } = age;
}
```

The compiler generates a constructor with those parameters. You can still add other constructors, but they must call the primary constructor using `this()`.

Primary constructors reduce boilerplate and make the code more concise. They are especially useful for simple data‑holding classes.

---

## 8.3 Object Initializers

**Object initializers** allow you to set properties or fields after calling a constructor, without needing multiple constructor overloads. They were introduced in C# 3.0 and are widely used.

```csharp
Person p = new Person { Name = "Alice", Age = 30 };
```

This is syntactic sugar for:

```csharp
Person temp = new Person(); // default constructor called
temp.Name = "Alice";
temp.Age = 30;
p = temp;
```

The actual temporary variable is invisible, and the assignment happens atomically (the object is fully initialized before being assigned to `p`).

Object initializers work with any accessible constructor. You can combine a parameterized constructor with an initializer:

```csharp
Person p = new Person("Bob") { Age = 25 };
```

Here, the constructor that takes a string is called first, then the property initializer runs.

### Collection Initializers

Similar syntax exists for collections that have an `Add` method:

```csharp
List<int> numbers = new List<int> { 1, 2, 3 };
Dictionary<string, int> ages = new Dictionary<string, int>
{
    { "Alice", 30 },
    { "Bob", 25 }
};
```

For dictionaries, the compiler translates each `{ key, value }` pair into a call to `Add(key, value)`.

### With Expressions (for Records)

Records support a special kind of initializer: the `with` expression, which creates a new instance with some properties changed (as seen in Chapter 7). This is not an object initializer per se, but it’s related to immutability and initialization.

---

## 8.4 The `required` Modifier (C# 11)

Starting with C# 11, you can mark properties or fields as **required**, meaning that they must be initialized either by a constructor or by an object initializer when creating an object. This is a more declarative way to enforce that certain data is provided.

```csharp
public class Person
{
    public required string Name { get; init; }
    public int Age { get; set; }
}

// Usage
var p = new Person { Name = "Alice", Age = 30 }; // OK
var p2 = new Person { Age = 30 }; // Compiler error: required property Name not set
```

The `required` modifier works with any property or field, but it’s most useful with `init` accessors to create immutable required data. The compiler ensures that every required member is set during object construction (either in a constructor or via an object initializer).

### Required and Constructors

If you have a constructor that sets a required property, you can mark the constructor with `[SetsRequiredMembers]` to tell the compiler that the constructor guarantees all required members are initialized. This avoids needing to use object initializers for those members.

```csharp
public class Person
{
    public required string Name { get; init; }
    public int Age { get; set; }

    [SetsRequiredMembers]
    public Person(string name, int age)
    {
        Name = name;
        Age = age;
    }
}

// Now you can do:
var p = new Person("Alice", 30); // No object initializer needed
```

Without `[SetsRequiredMembers]`, you’d still need to use an object initializer for `Name` even if the constructor sets it, because the compiler can’t verify that the constructor indeed sets all required members.

### Required with Record Types

Records can also use `required`:

```csharp
public record Person
{
    public required string FirstName { get; init; }
    public required string LastName { get; init; }
}
```

This ensures that when creating a `Person` with object initializer syntax, both names are provided.

---

## 8.5 Static Constructors

A **static constructor** is used to initialize static members of a class. It is called automatically before the first instance is created or any static member is accessed. It has no parameters and no access modifier.

```csharp
public class Configuration
{
    public static string ConfigValue { get; private set; }

    static Configuration()
    {
        Console.WriteLine("Static constructor called");
        ConfigValue = LoadFromFile();
    }

    private static string LoadFromFile() => "some value";
}
```

**Characteristics:**
- Runs exactly once per type, before any usage.
- Cannot be called directly; the runtime invokes it.
- If a static constructor throws an exception, the type becomes unusable for the lifetime of the application domain.
- Useful for setting up static data, reading configuration, etc.

Static constructors are also used with the **Singleton** pattern to ensure a single instance.

---

## 8.6 Putting It All Together: A Practical Example

Let’s build a `Product` class that demonstrates various constructor features: primary constructor, constructor chaining, `required` modifier, object initializers, and a static constructor for a product catalog.

```csharp
using System;
using System.Collections.Generic;

namespace ProductSystem
{
    // Primary constructor with required and init properties
    public class Product(string name, decimal price)
    {
        // Required properties (must be set)
        public required string Name { get; init; } = name 
            ?? throw new ArgumentNullException(nameof(name));

        public required decimal Price { get; init; } = price >= 0 ? price 
            : throw new ArgumentException("Price cannot be negative");

        // Optional property with default
        public string Category { get; set; } = "General";

        // Computed property
        public string DisplayName => $"{Name} ({Category}) - {Price:C}";

        // Constructor chaining example (if you want a parameterless option with defaults)
        public Product() : this("Unknown", 0) { }

        // Static constructor to initialize catalog
        private static List<Product> _catalog;

        static Product()
        {
            Console.WriteLine("Initializing product catalog...");
            _catalog = new List<Product>();
        }

        public static void AddToCatalog(Product product)
        {
            if (product == null) throw new ArgumentNullException(nameof(product));
            _catalog.Add(product);
        }

        public static void DisplayCatalog()
        {
            Console.WriteLine("\nProduct Catalog:");
            foreach (var p in _catalog)
            {
                Console.WriteLine($"  {p.DisplayName}");
            }
        }
    }

    class Program
    {
        static void Main()
        {
            // Using object initializer with required properties (must set Name and Price)
            var p1 = new Product
            {
                Name = "Laptop",
                Price = 1299.99m,
                Category = "Electronics"
            };

            // Using constructor that sets required members (with [SetsRequiredMembers] – if we had it)
            // Here we don't have that attribute, so we still need initializer if we use primary constructor directly?
            // Actually, primary constructor generates a constructor with parameters, so we can do:
            var p2 = new Product("Mouse", 29.99m)
            {
                Category = "Peripherals"
            };

            // Using parameterless constructor with object initializer
            var p3 = new Product
            {
                Name = "Desk",
                Price = 399.99m
            };

            // Add to static catalog
            Product.AddToCatalog(p1);
            Product.AddToCatalog(p2);
            Product.AddToCatalog(p3);

            // Display catalog (static constructor already ran)
            Product.DisplayCatalog();

            // Attempt to create invalid product (uncomment to see exception)
            // var invalid = new Product { Name = "Test", Price = -5m }; // throws
        }
    }
}
```

**Explanation:**

- The `Product` class uses a **primary constructor** with parameters `name` and `price`. These are used to initialize the required properties `Name` and `Price`, with validation in the initializer expressions.
- Properties are `required` and `init` – they must be set during construction and cannot be changed afterwards.
- There's a parameterless constructor that chains to the primary constructor with default values.
- A **static constructor** initializes the `_catalog` list and runs once before any static member is accessed.
- The `Main` method demonstrates creating objects with both object initializers and the primary constructor, then adding them to the static catalog.

---

## 8.7 Common Pitfalls and Best Practices

### 1. Always Validate Constructor Parameters

If your constructor receives data, validate it to ensure the object is created in a valid state. Throw appropriate exceptions (`ArgumentNullException`, `ArgumentException`) early.

### 2. Use Constructor Chaining to Avoid Duplication

When you have multiple constructors, have them all delegate to one primary constructor that does the actual work. This centralizes initialization logic.

### 3. Prefer Object Initializers for Optional Properties

Instead of creating many constructor overloads for every combination of optional properties, provide a constructor for required data and let callers use object initializers for optional ones. This is clean and maintainable.

### 4. Use `required` for Truly Mandatory Properties

If a property absolutely must be set by the caller, mark it `required`. This enforces it at compile time, reducing runtime errors.

### 5. Be Cautious with `[SetsRequiredMembers]`

Only apply this attribute to constructors that truly initialize all required members. If you later add a new required member and forget to update the constructor, the compiler won't warn you because the constructor claims to set them. Use it sparingly.

### 6. Static Constructors and Performance

Static constructors run once, but they can impact startup time if they do heavy work. Also, if a static constructor throws an exception, the type becomes unusable. Keep them simple and robust.

### 7. Avoid Doing Heavy Work in Constructors

Constructors should primarily initialize the object. Avoid calling virtual methods (they might be overridden in derived classes where the constructor hasn't finished running) or performing I/O that could fail. Use factory methods or separate initialization methods if needed.

### 8. Use Primary Constructors for Simple Classes

Primary constructors reduce boilerplate. They're great for DTOs, but for classes with complex logic, the traditional syntax may be clearer.

---

## 8.8 Chapter Summary

In this chapter, you've learned how objects come to life through constructors:

- **Default constructors** are provided by the compiler if you don't define any.
- **Parameterized constructors** let you require data at creation time.
- **Constructor chaining** with `this()` avoids code duplication.
- **Primary constructors** (C# 12) offer a concise syntax for classes with simple initialization.
- **Object initializers** allow setting properties after construction, reducing the need for multiple constructor overloads.
- The **`required` modifier** (C# 11) enforces that certain properties are set during initialization.
- **Static constructors** initialize static members once per type.

Understanding constructors gives you precise control over object creation, ensuring your objects are always in a valid state and your code remains clean and maintainable.

In the next chapter, **Inheritance & Polymorphism**, you'll build on these concepts to create hierarchies of classes, reuse code through inheritance, and write flexible code that can work with objects of different types through polymorphism. You'll learn about base classes, derived classes, virtual and override methods, abstract classes, and sealed classes – essential tools for designing extensible and organized systems.

**Exercises:**

1. Create a `BankAccount` class with a required property `AccountHolder`, a required `InitialBalance`, and an optional `AccountType`. Use a primary constructor and the `required` modifier. Add a static constructor that logs when the first account is created.
2. Write a `Rectangle` class with constructors that accept (width, height) and a separate constructor that accepts a single side (for a square). Use constructor chaining.
3. Experiment with object initializers: create a class with several properties and try different combinations of setting them via initializers.
4. Use the `required` modifier on a record and try to create an instance without setting a required property – observe the compiler error.
5. Add validation in a constructor that throws `ArgumentException` for invalid input, and see how it protects object state.

Now, get ready to explore the power of inheritance in Chapter 9!